In [1]:
# https://www.tensorflow.org/install/pip
%pip install nvidia-cudnn-cu11==8.6.0.163 tensorflow==2.12.* --quiet
# %pip install tensorflow_gnn --quiet
# %pip install tensorflow_io --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import feature_column
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
# import tensorflow_io as tfio
# import tensorflow_gnn as tfgnn
# from tensorflow_gnn import runner

2023-03-28 06:28:37.273607: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 06:28:38.378915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# verify install 
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-03-28 06:28:54.270806: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-28 06:28:57.372460: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-28 06:28:57.372750: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
print(tf.__version__)

2.12.0


In [5]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aada5d66-3f71-46b1-a1f6-7866112ec362;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (43ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (27ms)
:: resolution report :: resolve 761ms :: artifacts dl 74ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/28 06:29:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/28 06:29:23 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [6]:
# https://stackoverflow.com/questions/68036975/valueerror-shape-must-be-at-least-rank-3-but-is-rank-2-for-node-biasadd
# config for rank error in lstm
tf.config.threading.set_inter_op_parallelism_threads(16)
pd.set_option('display.max_columns', None)
tf.keras.backend.set_image_data_format("channels_last")

In [7]:
# Set Config
embedding_dim = 64
max_length = 6
sequence_length = 6
max_features = 10000
padding_type = 'post'
trunc_type = 'post'
training_portion = 0.8

In [8]:
df = spark.read.parquet("s3a://sapient-bucket-trusted/prod/tensor_sample_data/test_holdout/*").cache().toPandas()

In [9]:
df.count()

Trace             2598111
malicious         2598111
event_sequence    2598111
dtype: int64

In [10]:
len(df)

2598111

In [11]:
df.dtypes

Trace              int64
malicious          int32
event_sequence    object
dtype: object

In [12]:
# print(train_sequences[10])

In [13]:
# Data conversion - https://towardsdatascience.com/multi-class-text-classification-with-lstm-using-tensorflow-2-0-d88627c10a35
training_data = df['event_sequence'].tolist()

In [14]:
training_label = df['malicious'].tolist()

In [15]:
train_size = int(len(training_label) * training_portion)

train_set = training_data[0: train_size]
train_labels = training_label[0: train_size]
validation_set = training_data[train_size:]
validation_labels = training_label[train_size:]

In [16]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [17]:
tokenizer.fit_on_texts(training_data)

In [18]:
train_sequences = tokenizer.texts_to_sequences(train_set)
train_padded = tf.keras.utils.pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [19]:
validation_sequences = tokenizer.texts_to_sequences(validation_set)
validation_padded = tf.keras.utils.pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [20]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices([i for i in dataframe])
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
    return ds

In [63]:
train_ds = df_to_dataset(train_padded)

In [84]:
validation_ds = df_to_dataset(validation_padded)

In [64]:
train_sequences[0]

[2677810, 2677811, 2677812, 2677813, 2677814, 398041]

In [73]:
train_ds = tf.reshape(train_padded, (len(train_padded),6,1))

In [82]:
validation_ds = tf.reshape(validation_padded, (len(validation_padded),6,1))

In [30]:
vocab_size = len(set([x for l in train_sequences + validation_sequences for x in l]))

In [31]:
vocab_size

12463016

In [67]:
train_labels_ds = tf.ragged.constant(train_labels)
validation_labels_ds = tf.ragged.constant(validation_labels)

In [92]:
len(train_ds)

2078488

In [32]:
df.shape

(2598111, 3)

In [98]:
X = keras.Input(shape=[len(train_ds), 6], dtype=tf.int64, ragged=True)
l1 = keras.layers.Embedding(df.shape[0], 16)(X)
l2 = tf.reduce_sum(l1, axis=2) #To calculate the dense feature vector for each timestep.
l3 = keras.layers.LSTM(32, use_bias=False)(l2)
l4 = keras.layers.Dense(32, activation='sigmoid')(l3)
l5 = keras.layers.Activation(tf.nn.relu)(l4)
output = tf.keras.layers.Dense(1)(l5)

In [99]:
model = tf.keras.Model(X, output)

In [100]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 2078488, 6)]      0         
                                                                 
 embedding_10 (Embedding)    (None, 2078488, 6, 16)    41569776  
                                                                 
 tf.math.reduce_sum_7 (TFOpL  (None, 2078488, 16)      0         
 ambda)                                                          
                                                                 
 lstm_9 (LSTM)               (None, 32)                6144      
                                                                 
 dense_14 (Dense)            (None, 32)                1056      
                                                                 
 activation_5 (Activation)   (None, 32)                0         
                                                           

In [101]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [102]:
history = model.fit(
                x=train_ds,
                y=train_labels_ds,
                batch_size=32,
                epochs=num_epochs,
                verbose=1,
                callbacks=None,
                validation_split=0.0,
                validation_data=(validation_ds, validation_labels_ds),
                shuffle=True,
                class_weight=None,
                sample_weight=None,
                initial_epoch=0,
                steps_per_epoch=None,
                validation_steps=None,
                validation_batch_size=None,
                validation_freq=1,
                max_queue_size=10,
                workers=1,
                use_multiprocessing=False
                )

Epoch 1/10


ValueError: in user code:

    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/ec2-user/anaconda3/envs/python3/lib/p

In [62]:
type(train_ds)

tensorflow.python.framework.ops.EagerTensor

In [34]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(df.shape[0], embedding_dim, input_length=6),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(6, activation='softmax')
])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 6, 64)             166279104 
                                                                 
 dense_2 (Dense)             (None, 6, 64)             4160      
                                                                 
 dense_3 (Dense)             (None, 6, 6)              390       
                                                                 
Total params: 166,283,654
Trainable params: 166,283,654
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
num_epochs = 10
history = model.fit(train_padded, train_ds, epochs=num_epochs, 
                    validation_data=(validation_padded, validation_ds), 
                    verbose=2)

Epoch 1/10


2023-03-27 05:19:12.485880: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: required broadcastable shapes
2023-03-27 05:19:12.485942: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: required broadcastable shapes
	 [[{{node Equal}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'Equal' defined at (most recent call last):
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 724, in start
      self.io_loop.start()
    File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
# ds = tf.data.Dataset.from_tensor_slices(([i for i in df['event_sequence']], df['Trace'], df['malicious']))

In [ ]:
# Input data based on - https://www.tensorflow.org/tutorials/structured_data/feature_columns
train, test = train_test_split(df, test_size=0.2)

In [ ]:
train, val = train_test_split(train, test_size=0.2)

In [ ]:
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('malicious')
  ds = tf.data.Dataset.from_tensor_slices(([i for i in df['event_sequence']], df['malicious']))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
# A utility method to create a feature column
# and to transform a batch of data
def make_features(feature_column):  
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [ ]:
example_batch = next(iter(train))

In [ ]:
example_batch

In [ ]:
vocab_size = len(set([x[0] for x in df['event_sequence']] ))

In [ ]:
vocab_size

In [ ]:
# Vocabulary size and number of words in a sequence.
vocab_size = vocab_size
sequence_length = 6

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
text_ds = train_ds.map(lambda x, y: x)

In [ ]:
vectorize_layer.adapt(tf.reshape(text_ds, (len(text_ds),6,1)))

In [ ]:
for sequence_batch, label_batch in train_ds.take(1):
  print('A batch of sequences:', sequence_batch ) # list(feature_batch.keys()))
  print('A batch of targets:', label_batch )

In [ ]:
model = tf.keras.Sequential([
  #feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

In [ ]:
data = tf.ragged.constant([ [[940, 203, 668, 387, 790, 320, 939, 185],[315, 515, 791, 181, 939, 787]], 
                             [[564, 205], [820, 180, 993, 739]] ]) 

In [ ]:
# https://stackoverflow.com/questions/70399367/sparse-feature-vector-sequence-as-input-to-lstm
X = keras.Input(shape=[inputs.shape[0], inputs.shape[1]], dtype=tf.int64, ragged=True)
l1 = keras.layers.Embedding(inputs.shape[0], 16)(X)
l2 = tf.reduce_sum(l1, axis=2) #To calculate the dense feature vector for each timestep.
l3 = keras.layers.LSTM(32, use_bias=False)(l2)
l4 = keras.layers.Dense(32, activation='sigmoid')(l3)
l5 = keras.layers.Activation(tf.nn.relu)(l4)
output = tf.keras.layers.Dense(1)(l5)

In [ ]:
# Pad the sequences so they are all the same length
max_sequence_length = max([len(seq) for seq in sequences])
total_words = len(tokenizer.word_index) + 1
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [ ]:
print(model(data))

In [ ]:
model.predict(data)

In [ ]:
##Building input data
test01 = ([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]],'label1')
test02 = ([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[0,0,0,0]],'label2')
test03 = ([[1,1,1,1],[1,1,1,1],[1,1,1,1],[0,0,0,0],[1,1,1,1]],'label3')
test04 = ([[1,1,1,1],[0,0,0,0],[1,1,1,1],[1,1,1,1],[1,1,1,1]],'label1')
test_data = [test01,test02,test03,test04]


##DataFrame from data
columns = ['feature','label']
t_df = pd.DataFrame(data = test_data, columns = columns)

In [ ]:
t_df

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(([i for i in t_df['feature']], t_df['label']))

In [ ]:
dataset

In [ ]:
# https://stackoverflow.com/questions/70399367/sparse-feature-vector-sequence-as-input-to-lstm
X = keras.Input(shape=[t_df.shape[0], t_df.shape[1]], dtype=tf.int64, ragged=True)
l1 = keras.layers.Embedding(t_df.shape[0], 16)(dataset)
l2 = tf.reduce_sum(l1, axis=2) #To calculate the dense feature vector for each timestep.
l3 = keras.layers.LSTM(32, use_bias=False)(l2)
l4 = keras.layers.Dense(32, activation='sigmoid')(l3)
l5 = keras.layers.Activation(tf.nn.relu)(l4)
output = tf.keras.layers.Dense(1)(l5)

In [ ]:
t_df.shape